<h2>IS453 Financial Analytics</h2><h2>Lab 4 - Volatility, RAR, and Beta

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

**Documentation for new Pandas functions used**<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html<br>
https://numpy.org/doc/stable/reference/generated/numpy.reshape.html<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cov.html


## 1. Calculate annualized volatility

Read 5 years of data for SPY between 2017-2021. Extract the adjusted close prices.



In [2]:
# sample code
tickers = ['SPY']

start_date = '2016-12-30'
end_date = '2022-01-01' 

data1 = yf.download(tickers, start= start_date, end= end_date)

stock_price_5y = data1[['Adj Close']]
stock_price_5y

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2016-12-30,195.791016
2017-01-03,197.288879
2017-01-04,198.462601
2017-01-05,198.304886
2017-01-06,199.014389
...,...
2021-12-27,456.750977
2021-12-28,456.377686
2021-12-29,456.961487


**Example showing how the pandas df.shift() method works**

In [3]:
# sample code
# the output gets shifted by forward by 1 unit of date period (index)
test_data = stock_price_5y[['Adj Close']]
test_data['shifted']= test_data.shift(1)
test_data

,Adj Close,shifted
Date,,
2016-12-30,195.791016,NaN
2017-01-03,197.288879,195.791016
2017-01-04,198.462601,197.288879
2017-01-05,198.304886,198.462601
2017-01-06,199.014389,198.304886
...,...,...
2021-12-27,456.750977,450.377136
2021-12-28,456.377686,456.750977
2021-12-29,456.961487,456.377686


In [4]:
# to find the ratio of the change in the current price to the previous price
test_data['ratio'] = test_data['Adj Close']/test_data['Adj Close'].shift(1)
test_data

,Adj Close,shifted,ratio
Date,,,
2016-12-30,195.791016,NaN,NaN
2017-01-03,197.288879,195.791016,1.007650
2017-01-04,198.462601,197.288879,1.005949
2017-01-05,198.304886,198.462601,0.999205
2017-01-06,199.014389,198.304886,1.003578
...,...,...,...
2021-12-27,456.750977,450.377136,1.014152
2021-12-28,456.377686,456.750977,0.999183
2021-12-29,456.961487,456.377686,1.001279


**Exercise Q1: Calculate the annualized historical volatility for SPY**

Calculate for periods of the most recent 20 days, 3 months, 1 year (2021) and 5 years (the full data set).<BR>
Note that the calculations must be based on log returns from prices as Rn = log(Cn/Cn-1)

The formula for annualising historical volatility is in the lecture slides.<BR>
  You can assume that there are 252 trading days in a year.

*Hint: For defined periods of data such as x-months or x-years, use .loc to select data ranges to subset the prices. For the 20-day period, it refers to twenty trading days, not calendar days.  So use tail or .iloc instead to select the range.*

 - Which period has the highest annualized historical volatility? <br>
 - Why do you think that period has the highest?</b></font>

**Example: 20-day HV**

In [5]:
# sample code 

# get the most recent 20 days
spy_20d = stock_price_5y['Adj Close'].iloc[-20:]

# take the log of price change(returns) and then apply std()
# default for Pandas std() is sample formula
spy_20d_hv = np.log(spy_20d / spy_20d.shift(1)).std()

# to annualize, multiply by sqrt(252)
ann_spy_20d_hv = spy_20d_hv * np.sqrt(252)
print(f'{ann_spy_20d_hv:.1%}')

16.3%


**Find the HV for other periods**

In [6]:
# your code here
# Note: use the loc method to subset the prices for the period and then apply the formula to find HV

#3 months - Oct to Dec 2021
spy_3m = stock_price_5y['Adj Close'].loc['2021-10-01':'2022-01-01']
spy_3m_hv = np.log(spy_3m / spy_3m.shift(1)).std()
ann_spy_3m_hv = spy_3m_hv * np.sqrt(252)
print(f'3-month: {ann_spy_3m_hv:.1%}')


#1 year - for all 2021
spy_1y = stock_price_5y['Adj Close'].loc['2021-01-01':'2022-01-01']
spy_1y_hv = np.log(spy_1y / spy_1y.shift(1)).std()
ann_spy_1y_hv = spy_1y_hv * np.sqrt(252)
print(f'1-year: {ann_spy_1y_hv:.1%}')

#5 years
spy_5y = stock_price_5y['Adj Close']
spy_5y_hv = np.log(spy_5y / spy_5y.shift(1)).std()
ann_spy_5y_hv = spy_5y_hv * np.sqrt(252)
print(f'5-year: {ann_spy_5y_hv:.1%}')



3-month: 13.8%
1-year: 12.9%
5-year: 19.0%


- Your answer here

## 2. Calculate risk adjusted returns

In [7]:
tickers = ['SPY','TLT','EFA']

start_date = '2020-12-31'
end_date = '2022-01-01' 

data2 = yf.download(tickers, start= start_date, end= end_date)

stocks_price1 = data2['Adj Close']
stocks_price1

[*********************100%%**********************]  3 of 3 completed


,EFA,SPY,TLT
Date,,,
2020-12-31,64.694130,353.106628,140.378098
2021-01-04,65.031090,348.299408,140.208939
2021-01-05,65.704994,350.698273,139.167725
2021-01-06,66.458679,352.794922,136.310791
2021-01-07,66.529617,358.036682,135.109329
...,...,...,...
2021-12-27,72.352768,456.750946,134.545441
2021-12-28,72.380196,456.377686,134.012192
2021-12-29,72.352768,456.961517,132.548233


**Exercise Q2: Calculate the Sharpe ratio for each fund**

For the risk-adjusted return, assume that the daily risk-free rate is 0.0001.<br>

- Which fund has the highest RAR
- Which fund has the lowest RAR

In [8]:
# sample code

daily_risk_free_rate = 0.0001

# create the percent returns and delete blank first row
pct_returns1 = stocks_price1.pct_change().dropna()

# find the average daily return values for the instruments
avg_daily_ret = pct_returns1.mean()

# create a new dataframe and add required columns to compute Sharpe ratio
stocks_df_RAR = pd.DataFrame(pct_returns1)

# add the risk free rate column in the df
stocks_df_RAR['RiskFree_Rate'] = daily_risk_free_rate

# compute the average risk free rate value
avg_rf_ret = stocks_df_RAR['RiskFree_Rate'].mean()

# add the excess return columns for each ETF
stocks_df_RAR['Excess_ret_SPY'] = stocks_df_RAR["SPY"] - stocks_df_RAR['RiskFree_Rate']
stocks_df_RAR['Excess_ret_TLT'] = stocks_df_RAR["TLT"] - stocks_df_RAR['RiskFree_Rate']
stocks_df_RAR['Excess_ret_EFA'] = stocks_df_RAR["EFA"] - stocks_df_RAR['RiskFree_Rate']

# the constructed df for RAR calculations
stocks_df_RAR

,EFA,SPY,TLT,RiskFree_Rate,Excess_ret_SPY,Excess_ret_TLT,Excess_ret_EFA
Date,,,,,,,
2021-01-04,0.005209,-0.013614,-0.001205,0.0001,-0.013714,-0.001305,0.005109
2021-01-05,0.010363,0.006887,-0.007426,0.0001,0.006787,-0.007526,0.010263
2021-01-06,0.011471,0.005978,-0.020529,0.0001,0.005878,-0.020629,0.011371
2021-01-07,0.001067,0.014858,-0.008814,0.0001,0.014758,-0.008914,0.000967
2021-01-08,0.009330,0.005697,-0.003228,0.0001,0.005597,-0.003328,0.009230
...,...,...,...,...,...,...,...
2021-12-27,0.007770,0.014152,0.002424,0.0001,0.014052,0.002324,0.007670
2021-12-28,0.000379,-0.000817,-0.003963,0.0001,-0.000917,-0.004063,0.000279
2021-12-29,-0.000379,0.001279,-0.010924,0.0001,0.001179,-0.011024,-0.000479


In [9]:
# sample code

# calculate sharpe ratio for SPY
sharpe_SPY = (avg_daily_ret['SPY'] - avg_rf_ret)/stocks_df_RAR['Excess_ret_SPY'].std()
ann_sharpe_SPY = sharpe_SPY * np.sqrt(252)
print(f'Annualised Sharpe Ratio SPY = {ann_sharpe_SPY:.3f}')


Annualised Sharpe Ratio SPY = 1.816


In [10]:
# your code here

# calculate sharpe ratio for TLT
sharpe_TLT = (avg_daily_ret['TLT'] - avg_rf_ret)/stocks_df_RAR['Excess_ret_TLT'].std()
ann_sharpe_TLT = sharpe_TLT * np.sqrt(252)
print(f'Annualised Sharpe Ratio TLT = {ann_sharpe_TLT:.3f}')

# calculate sharpe ratio for EFA
sharpe_EFA = (avg_daily_ret['EFA'] - avg_rf_ret)/stocks_df_RAR['Excess_ret_EFA'].std()
ann_sharpe_EFA = sharpe_EFA * np.sqrt(252)
print(f'Annualised Sharpe Ratio EFA = {ann_sharpe_EFA:.3f}')

Annualised Sharpe Ratio TLT = -0.452
Annualised Sharpe Ratio EFA = 0.714


## 3. Calculate beta

Beta is a measure of the volatility of an individual asset in comparison to the volatility of the entire market as measured by a benchmark.<BR>
We will use the covariance to calculate.
  
**Exercise Q3: Calculate the 1 year beta for Apple stocks with the SPY market benchmark.**

- Does AAPL have higher or lower risk sensitivity compared to the S&P 500 index (SPY)?

In [11]:
# sample code
# convert the price series to percent returns and drop null values before computing beta

tickers = ['AAPL','SPY']

start_date = '2020-12-31'
end_date = '2022-01-01'

data3 = yf.download(tickers, start= start_date, end= end_date)
stocks_price2 = data3['Adj Close']

stocks_price2_returns = stocks_price2.pct_change().dropna()
stocks_price2_returns

[*********************100%%**********************]  2 of 2 completed


,AAPL,SPY
Date,,
2021-01-04,-0.024719,-0.013614
2021-01-05,0.012363,0.006887
2021-01-06,-0.033661,0.005979
2021-01-07,0.034123,0.014857
2021-01-08,0.008631,0.005698
...,...,...
2021-12-27,0.022975,0.014152
2021-12-28,-0.005767,-0.000817
2021-12-29,0.000502,0.001279


In [12]:
# sample code
# find the covariance, the default for Pandas cov() is sample formula
stocks_price2_returns.cov()

,AAPL,SPY
AAPL,0.000250,0.000089
SPY,0.000089,0.000067


Calculate beta using the formula covar(APPL,SPY)/var(SPY)

*Hint: use .loc to extract the APPL-SPY covariance value for the calculation*

In [13]:
# your code here
aapl_spy_covar = stocks_price2_returns.cov().loc['AAPL','SPY']
benchmark_variance = stocks_price2_returns.cov().loc['SPY','SPY']
beta = aapl_spy_covar/benchmark_variance
beta.round(3)

1.325

- Answer here

## DIY


### 4. Portfolio construction analysis
You will construct portfolios containing five stocks, selected from the 15 highest weighted S&P 500 stocks.<BR>
Use 5 years worth of data for your analysis - from 1 Jan 2017 through 31 Dec 2021.<BR>
S&P 500 component stocks listed at: https://www.slickcharts.com/sp500

**DIY Q1: Which five stocks would you use to create a “low volatility” portfolio?**

*Hint: For simplicity, only consider the stocks' volatilities. You can ignore their correlations.*<BR>
*Hint: use .sort_values()*<BR>
*Hint: the ticker symbol for Berkshire Hathaway Inc. is "BRK-B"*

In [119]:
# your code here
tickers = ['AAPL','NVDA','MSFT','AMZN','META','GOOGL','AVGO','TSLA','GOOG','BRK-B','JPM','LLY','V','UNH','XOM']
start_date = '2017-01-01'
end_date = '2022-01-01' 

data = yf.download(tickers, start= start_date, end= end_date)

stock_price = data[['Adj Close']]['Adj Close']

hv_5y = np.log(stock_price / stock_price.shift(1)).std()
ann_5y_hv = hv_5y * np.sqrt(252)
ann_5y_hv = ann_5y_hv.sort_values()
ann_5y_hv[:5]


[*********************100%%**********************]  15 of 15 completed


BRK-B    0.215351
V        0.266078
GOOG     0.270058
GOOGL    0.270831
MSFT     0.275065
dtype: float64

Answer: <b>I would use MSFT, GOOGL, GOOG, V, and BRK.B. </b>

**DIY Q2: Which five stocks would you use to create a “best RAR” portfolio?**

Assume a daily risk-free rate of 0.0001.

In [112]:
# your code here
def prepare_df(tickers, df):    
    for ticker in tickers:
        df['Excess_ret_'+ticker] = df[ticker] - df['risk_free_rate']
 
def get_top_5_sharpe(tickers, df, avg_daily_ret, avg_rf_ret):
    dictionary = {}
    for ticker in tickers:
        sharpe = (avg_daily_ret[ticker] - avg_rf_ret)/df['Excess_ret_'+ticker].std()
        ann_sharpe = sharpe * np.sqrt(252)
        dictionary[ticker] = ann_sharpe    
    return pd.Series(dictionary).sort_values(ascending=False)[0:5]

daily_risk_free_rate = 0.0001
pct_returns = stock_price.pct_change().dropna()
avg_daily_ret = pct_returns.mean()
stocks_df_RAR = pd.DataFrame(pct_returns)
stocks_df_RAR['risk_free_rate'] = daily_risk_free_rate
avg_rf_ret = stocks_df_RAR['risk_free_rate'].mean()

prepare_df(tickers, stocks_df_RAR)
print(get_top_5_sharpe(tickers, stocks_df_RAR, avg_daily_ret, avg_rf_ret))



MSFT    1.325050
TSLA    1.319277
AAPL    1.301532
NVDA    1.230854
AMZN    1.073643
dtype: float64


Answer: <b> I would use MSFT, TSLA, AAPL, NVDA and AMZN. </b>

**DIY Q3: Which five stocks would you use to create a “low market risk sensitivity” portfolio?**

*Hint: make sure SPY is one of your five stocks.*

In [113]:
# your code here
def get_5_lowest_risk(tickers, pct_returns, benchmark_variance):
    dictionary = {}
    for ticker in tickers:
        if ticker != 'SPY':
            spy_covar = pct_returns.cov().loc[ticker,'SPY']
            beta = spy_covar/benchmark_variance
            dictionary[ticker] = beta.round(3)
    return pd.Series(dictionary).sort_values()[:5]

tickers = ['AAPL','NVDA','MSFT','AMZN','META','GOOGL','AVGO','TSLA','GOOG','BRK-B','JPM','LLY','V','UNH','XOM','SPY']
start_date = '2017-01-01'
end_date = '2022-01-01' 

data = yf.download(tickers, start= start_date, end= end_date)

stock_price = data[['Adj Close']]['Adj Close']
pct_returns = stock_price.pct_change().dropna()
benchmark_variance = pct_returns.cov().loc['SPY','SPY']
print(get_5_lowest_risk(tickers, pct_returns, benchmark_variance))

[*********************100%%**********************]  16 of 16 completed
LLY      0.723
BRK-B    0.935
AMZN     0.953
UNH      1.039
XOM      1.046
dtype: float64


Answer: <b> I would use LLY, BRK.B, AMZN, UNH and XOM. </b>

**DYI Q4:**

It is troublesome to have to try to determine the last business day of the previous year by trial and error. <BR>
There should a way to determine that date using Python functions. Let's use GenAI to help us to learn how.

Use Generative AI to help you write the Python code to return the string of the last weekday for a given year. <BR>
The results of the code for different years should be:<BR>
2022 = '2022-12-30'<BR>
2023 = '2023-12-29'<BR>
2024 = '2024-12-31'

In [ ]:
# your successful GenAI prompts here:

# I need you to create a simple utility function, get_last_weekday(year) for me. This function takes in a year, and returns the last weekday of the year. 

# The output should be as follows: 
# 2022 = '2022-12-30'
# 2023 = '2023-12-29'
# 2024 = '2024-12-31'

In [116]:
# your code here

def get_last_weekday(year):
    last_day = pd.date_range(start=f'{year}-12-01', end=f'{year}-12-31', freq='B')[-1]
    return last_day.strftime('%Y-%m-%d')

print(get_last_weekday(2022))
print(get_last_weekday(2023))
print(get_last_weekday(2024))


2022-12-30
2023-12-29
2024-12-31
